First, import the dataset and convert it into a pandas dataframe:
----------

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# Helper functions used in other parts of the code
import HelperModules

In [ ]:
ds = xr.open_dataset('MyChallengePaleo/T2m_R1_ym_1stMill.nc')
T2m_R1 = ds.to_dataframe()['T2m']
T2m_R1

Seems like we are using a pandas MultiIndex, so let's learn a little bit about that:
-----

In [ ]:
T2m_R1.index.names

Now, we want to plot some of the data. How do we do that? Let's first see what times are available... and find the first available time.
--------

In [ ]:
print(T2m_R1.index.get_level_values('time'))
print(min(list(T2m_R1.index.get_level_values('time'))))

We use the "xs" tool to get a slice of the data corresponding to the earliest time:
--------
And we put it into a little function called `getTimeSlice` (in HelperModules) so that we can spot-check different years...

In [ ]:
slice0 = HelperModules.getTimeSlice(T2m_R1,1)
slice0

In [ ]:
# We could turn this into a flat dataframe, but in the end I think we do not need to do this.
# Let's keep the code around in any case.

#slice0_df = slice0.reset_index(inplace=False)
#slice0_df

Let's take the MultiIndexed data that we hvae and use "unstack" to turn it into a standard python dataframe, where rows are latitude and columns are longitude.
-----
We will also reset the longitude to run from -180 to +180, in order to get it working more smoothly with the plotting.

In [ ]:
# "Unstack" the multiindex data:
unstacked = HelperModules.Unstack(slice0)
unstacked

Okay, this seems like something we can plot! Let's now try to figure out how to plot this.
-------
For this we will use **cartopy** (in python3, on a mac). I had a bit of difficulty finding a recipe that worked, here it is below:
```
brew extract --version=5.2.0 proj $USER/local-tap
brew install proj@5.2.0
pip3 install cartopy
```
Now we use matplotlib and cartopy to make the world-map plots.
-------

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

Now let's try to plot our data on a map like this.
------------

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Make the mesh grid
lon = unstacked.columns
lat = unstacked.index
m_lon, m_lat = np.meshgrid(lon, lat)

the_contour = ax.contourf(m_lon, m_lat,unstacked.values,transform=ccrs.PlateCarree())
fig.colorbar(the_contour)
ax.set_global()
ax.coastlines()

Great - now let's put it into a function!
------------
You can see the function `plotTimeSlice(...)` in the HelperModules file.

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
HelperModules.plotTimeSlice(T2m_R1,1,fig,ax,vmin=200,vmax=350)

Getting to know the data better
=================
What datasets are available to us? 

| File                      | Shape | Description and Notes |
| :----                     | :---  | :----                 |
| T2m_R1_ym_1stMill.nc      | (192,96,999) lon, lat, time | Annual means for the simulation periods 1–999 AD. 2m-temperatures. |
| T2m_R2_ym_1stMill.nc      | | Same thing, but a different simulation. The same set of changes in external forcing drives these two simulations, but they differ slightly in their initial conditions, reflecting our ignorance of the actual conditions at the time. |
| Solar_forcing_1st_mill.nc | (1,1,999) lev, time, x | Solar forcing. The curve shows the forcing time series of the Total Solar Irradiance changes in units of Wm-2. |
| Volc_Forc_AOD_1st_mill.nc | | Volcanic forcing. The files includes the so-called Aerosol Optical Depth (AOD) as time series. High values indicate a high sulfate load in the stratosphere, leading to a reflection and absorption of incoming short wave radiation in the stratosphere with the net-effectof a decrease in global mean temperatures.  |

Let us load these files to get a better idea (we have already loaded T2m_R1).

Solar Forcing
------

In [ ]:
ds = xr.open_dataset('MyChallengePaleo/Solar_forcing_1st_mill.nc')
TSI = ds.to_dataframe()['TSI']
# "lev" and "x" seem to be meaningless, so let's get rid of them and turn the series into a dataframe.
TSI = TSI.xs((1.0)).xs(0,level='x').to_frame()
TSI = TSI.reset_index()
TSI['time_yr'] = np.round(TSI['time']/10000)
TSI

Volcanic Forcing
-----------

In [ ]:
ds = xr.open_dataset('MyChallengePaleo/Volc_Forc_AOD_1st_mill.nc')
AOD = ds.to_dataframe()['AOD']
AOD = AOD.xs((1.0)).xs(0,level='x').to_frame()
AOD = AOD.reset_index()
AOD['time_yr'] = np.round(AOD['time']/10000)
AOD

Let's plot both!
------
In the first plot, we plot three things: solar forcing, the rolling-average of the solar forcing (to remove the 11-year cycle), and the volcanic forcing.

We will also plot a few dots on the first plot, to be able to isolate certain years of interest for us.

Below that, we plot maps of the two years of interest, so we can compare them visually.

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))

# Let's make a little snapshot-picker piece of code
snapshot_year = 517.0
snapshot_aod = AOD[AOD['time_yr'] == snapshot_year]
snapshot_tsi = TSI[TSI['time_yr'] == snapshot_year]

snapshot_year_b = 532.0
snapshot_aod_b = AOD[AOD['time_yr'] == snapshot_year_b]
snapshot_tsi_b = TSI[TSI['time_yr'] == snapshot_year_b]

# Solar forcing
color = 'gray'
ax1.plot(TSI['time_yr'],TSI['TSI'],label='solar irradiance',color=color)
ax1.scatter(snapshot_tsi['time_yr'],snapshot_tsi['TSI'],color='tab:red')
ax1.scatter(snapshot_tsi_b['time_yr'],snapshot_tsi_b['TSI'],color='tab:green')
ax1.set_xlabel('time (year)')
ax1.set_ylabel('Solar irradiance [Wm$^{-2}$]',color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Rolling-average solar forcing
ax1.plot(TSI['time_yr'],TSI['TSI'].rolling(window=11,center=True).mean(),label='solar irradiance (11-yr mean)',color='tab:red')

# Volcanic forcing
color = 'tab:blue'
ax2 = ax1.twinx()
ax2.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color=color)
ax2.scatter(snapshot_aod['time_yr'],snapshot_aod['AOD'],color='tab:red')
ax2.scatter(snapshot_aod_b['time_yr'],snapshot_aod_b['AOD'],color='tab:green')
ax2.set_ylabel('Aerosol Optical Depth',color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Two snapshots, representing the two chosen years for visual comparison.
figcompare = plt.figure(figsize=(20, 4))
ax_compare = figcompare.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax_compare2 = figcompare.add_subplot(1, 2, 2, projection=ccrs.PlateCarree())

HelperModules.plotTimeSlice(T2m_R1,snapshot_year,figcompare,ax_compare,vmin=210,vmax=315)
HelperModules.plotTimeSlice(T2m_R1,snapshot_year_b,figcompare,ax_compare2,vmin=210,vmax=315)

Engineering some Features from the Temperature Data
-------------
Start with some basic things, like global average temperature. We can use this to show that short-term trends of the global average can reveal spikes revealing volcanic activity.


Spatial average (global temperature average)
-----------

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 1, 1)
spacial_averaged = T2m_R1.groupby('time').mean()
p1 = ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='gray')
p2 = ax.plot(TSI['time_yr'],0.3*(-1363.0+TSI['TSI'].rolling(window=11,center=True).mean()),label='solar irradiance (11-yr mean), arb. units',color='red')
ax.set_ylabel('Aerosol Optical Depth',color='gray')
ax.set_ylim(0,0.9)

ax2 = ax.twinx()
p3 = ax2.plot(np.round(spacial_averaged.index/10000),list(spacial_averaged.values),label='Average global temperature')
ax2.set_ylabel('Average global temperature',color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim(274.5,278.3)

labs = [l.get_label() for l in p1+p2+p3]
ax.legend(p1+p2+p3, labs, loc=3)

plt.show()

Temporal average (local temperature average over time)
-----
Let's see if we can improve our visualization by establishing a baseline temperature at each longitude and latitude:

In [ ]:
temporal_averaged = T2m_R1.groupby(['lat','lon']).mean()
temporal_averaged

In [ ]:
figcompare = plt.figure(figsize=(10, 5))
ax = figcompare.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
HelperModules.plotMap(temporal_averaged,figcompare,ax)

Now let's try to plot the anomaly map of a given timeSlice:
---------

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
#plotMap(getTimeSlice(525) - temporal_averaged,ax)
HelperModules.plotMap(temporal_averaged-HelperModules.getTimeSlice(T2m_R1,525),fig,ax,vmin=-5,vmax=5)

Now let's try to plot a bunch of time slices in sequence
----------
This will _maybe_ help us to visualize changes over a relatively short time-span.

With this, starting in year 525, we can really see how the northern hemisphere gets cold in response to a surge in volcanic activity.

In [ ]:
import matplotlib.gridspec as gridspec

nslices = 12
start_year = 525

# First plot: a sequence of time slices in the years of interest
fig = plt.figure(figsize=(14, 10),constrained_layout=True)
spec = gridspec.GridSpec(ncols=3, nrows=(1+nslices//3), figure=fig)

axes = []
for i in range(nslices) :
    axes.append(fig.add_subplot(spec[i//3, i%3], projection=ccrs.PlateCarree()))
    this_timeslice = HelperModules.getTimeSlice(T2m_R1,start_year+i)
    the_contour = HelperModules.plotMap(this_timeslice - temporal_averaged,fig,axes[-1],vmin=-5,vmax=5)
    axes[-1].title.set_text('Year %s'%(start_year+i))



# Second plot: the solar forcing, vocanic activity and average temperature for that time period.
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
spacial_averaged = T2m_R1.groupby('time').mean()
p1 = ax.plot(AOD['time_yr'],AOD['AOD'],label='volcanic AOD',color='gray')
p2 = ax.plot(TSI['time_yr'],0.5*(-1365.0+TSI['TSI']),label='solar irradiance (arb units)',color='orange')
ax2 = ax.twinx()
p3 = ax2.plot(np.round(spacial_averaged.index/10000),list(spacial_averaged.values),label='avg global temperature')

labs = [l.get_label() for l in p1+p2+p3]
ax.legend(p1+p2+p3, labs, loc=0)

plt.xlim(start_year-1,start_year+13)
plt.show()

Let's Engineer some Features! First let's get a specific slice of latitude (e.g. southern hemisphere)
----
The function `getLatSlice(...)` lives in the HelperModules file.

In [ ]:
HelperModules.getLatSlice(T2m_R1,-999,-60)